In [1]:
import pandas as pd
# import tools_vk
df = pd.read_csv('result1.csv')


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   index                     820 non-null    int64  
 1   id                        820 non-null    int64  
 2   from_id                   820 non-null    int64  
 3   owner_id                  820 non-null    int64  
 4   date                      820 non-null    int64  
 5   marked_as_ads             820 non-null    int64  
 6   post_type                 820 non-null    object 
 7   text                      804 non-null    object 
 8   attachments               812 non-null    object 
 9   is_favorite               820 non-null    bool   
 10  post_source.type          820 non-null    object 
 11  comments.count            820 non-null    int64  
 12  comments.can_post         820 non-null    int64  
 13  comments.groups_can_post  820 non-null    bool   
 14  likes.coun